# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski

### Instructions
- Sign up for RunPod here: https://runpod.io/?ref=n8yfwyum
    - Note: That's my personal referral link. Please don't use it if we are mortal enemies.

- Click *Deploy* on either `SECURE CLOUD` or `COMMUNITY CLOUD`

- Follow the rest of the instructions in this video: https://www.youtube.com/watch?v=7m__xadX0z0#t=5m33.1s

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [ ]:
# BUILD ENV
!pip install numpy==1.23.1
!pip install pytorch-lightning==1.7.6
!pip install csv-logger
!pip install torchmetrics==0.11.1
!pip install torch-fidelity==0.3.0
!pip install albumentations==1.1.0
!pip install opencv-python==4.7.0.72
!pip install pudb==2019.2
!pip install omegaconf==2.1.1
!pip install pillow==9.4.0
!pip install einops==0.4.1
!pip install transformers==4.25.1
!pip install kornia==0.6.7
!pip install diffusers[training]==0.3.0
!pip install captionizer==1.0.1
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .
!pip install huggingface_hub
!pip install gitpython


# Download the 1.5 model from hugging face
You can also provide your own v1.* model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as dreambooth_runpod_joepenna.ipynb

In [ ]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

# Regularization Images (Skip this section if you are uploading your own or using the provided images)

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [ ]:
# GENERATE 200 images - Optional
self_generated_files_prompt = "person" #@param {type:"string"}
self_generated_files_count = 200 #@param {type:"integer"}

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt {self_generated_files_prompt}

dataset=self_generated_files_prompt

!mkdir -p regularization_images/{dataset}
!mv outputs/txt2img-samples/*.png regularization_images/{dataset}

In [ ]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset
!apt-get install -y zip
!zip -r regularization_images.zip regularization_images/{dataset}

# Download pre-generated regularization images
We've created the following image sets

* `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
* `man_unsplash` - pictures from various photographers
* `person_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
* `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
* `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0

`person_ddim` is recommended for training people


In [ ]:
#Download Regularization Images

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

In [ ]:
#@markdown Add here the URLs to the images of the subject you are adding
urls = [
 "https://i.imgur.com/test1.png",
 "https://i.imgur.com/test2.png",
 "https://i.imgur.com/test3.png",
 "https://i.imgur.com/test4.png",
 "https://i.imgur.com/test5.png",
 # You can add additional images here -- about 20-30 images in different
]

In [ ]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_images"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# If you are training a person's face, set this to True
i_am_training_a_persons_face = False

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

# This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName"

# 0 Saves the checkpoint when max_training_steps is reached.
# 250 saves the checkpoint every 250 steps as well as when max_training_steps is reached.
save_every_x_steps = 0

reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --debug False \
 --max_training_steps {max_training_steps} \
 --token "{token}" \
 --training_model "model.ckpt" \
 --training_images "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --regularization_images "{reg_data_root}" \
 --class_word "{class_word}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"